In [3]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, parse_qs, urlparse

# 2-2에서 만든 함수 재사용
def download_one_episode(title, no, url):
    req_header = {
        'referer': url,
        "user-agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/104.0.0.0 Safari/537.36"
        )
    }

    res = requests.get(url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        img_tags = soup.select("img[src*='IMAG01']")

        base_dir = os.path.join("img", title, str(no))
        os.makedirs(base_dir, exist_ok=True)

        for idx, img_tag in enumerate(img_tags, 1):
            img_url = img_tag['src']
            img_data = requests.get(img_url, headers=req_header).content

            ext = os.path.splitext(img_url)[-1]
            filename = f"{idx:03d}{ext}"
            file_path = os.path.join(base_dir, filename)

            with open(file_path, 'wb') as file:
                file.write(img_data)
                print(f"✓ 저장됨: {file_path} ({len(img_data):,} bytes)")
    else:
        print(f"[Error] 요청 실패: {res.status_code}")

# 2-3. 전체 회차 처리 함수
def download_all_episode(title, episode_url):
    headers = {
        "user-agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/104.0.0.0 Safari/537.36"
        )
    }

    res = requests.get(episode_url, headers=headers)
    if not res.ok:
        print(f"[Error] 회차 목록 페이지 요청 실패: {res.status_code}")
        return

    soup = BeautifulSoup(res.text, 'html.parser')

    # 회차 <a href="..."> 추출
    detail_links = soup.select('td.title a[href^="/webtoon/detail"]')

    episode_urls = []
    seen = set()  # 중복 제거

    for a in detail_links:
        href = a.get('href')
        full_url = urljoin(episode_url, href)
        parsed = urlparse(full_url)
        query = parse_qs(parsed.query)
        no_list = query.get('no')

        if no_list:
            no = int(no_list[0])
            if no not in seen:
                seen.add(no)
                episode_urls.append((no, full_url))

    # 내림차순 정렬 (최신 회차부터)
    episode_urls.sort(reverse=True)

    print(f"[INFO] 총 {len(episode_urls)}개 회차 발견됨")

    # 각 회차별 다운로드 실행
    for no, url in episode_urls:
        print(f"\n▶ {title} {no}화 다운로드 중...")
        download_one_episode(title, no, url)


In [5]:
download_all_episode('배달왕', 'https://comic.naver.com/webtoon/list?titleId=823933')

[INFO] 총 0개 회차 발견됨
